In [1]:
using BenchmarkTools

In [25]:
do_stuff1(x) = (x .* 2) .^ 2
do_stuff2(x) = [i < 1_000 ? i*2 : i*3 for i in x]

do_stuff2 (generic function with 1 method)

In [26]:
do_stuff1(1:100_000)

100000-element Array{Int64,1}:
           4
          16
          36
          64
         100
         144
         196
         256
         324
         400
         484
         576
         676
           ⋮
 39991200484
 39992000400
 39992800324
 39993600256
 39994400196
 39995200144
 39996000100
 39996800064
 39997600036
 39998400016
 39999200004
 40000000000

In [27]:
do_stuff2(1:100_000)

100000-element Array{Int64,1}:
      2
      4
      6
      8
     10
     12
     14
     16
     18
     20
     22
     24
     26
      ⋮
 299967
 299970
 299973
 299976
 299979
 299982
 299985
 299988
 299991
 299994
 299997
 300000

In [28]:
@btime 1:100 .|> x->do_stuff1(1:(10_000+x))

  1.879 ms (201 allocations: 7.68 MiB)


100-element Array{Array{Int64,1},1}:
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399360256, 399440196, 399520144, 399600100, 399680064, 399760036, 399840016, 399920004, 400000000, 400080004]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399440196, 399520144, 399600100, 399680064, 399760036, 399840016, 399920004, 400000000, 400080004, 400160016]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399520144, 399600100, 399680064, 399760036, 399840016, 399920004, 400000000, 400080004, 400160016, 400240036]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399600100, 399680064, 399760036, 399840016, 399920004, 400000000, 400080004, 400160016, 400240036, 400320064]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399680064, 399760036, 399840016, 399920004, 400000000, 400080004, 400160016, 400240036, 400320064, 400400100]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  399760036, 399840016, 399920004, 400000000, 400080004, 400160016, 400240036, 400320064, 400400100, 400480144]
 [4

In [29]:
@btime 1:100 .|> x->do_stuff1(1:(100_000+x))

  13.014 ms (201 allocations: 76.34 MiB)


100-element Array{Array{Int64,1},1}:
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39993600256, 39994400196, 39995200144, 39996000100, 39996800064, 39997600036, 39998400016, 39999200004, 40000000000, 40000800004]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39994400196, 39995200144, 39996000100, 39996800064, 39997600036, 39998400016, 39999200004, 40000000000, 40000800004, 40001600016]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39995200144, 39996000100, 39996800064, 39997600036, 39998400016, 39999200004, 40000000000, 40000800004, 40001600016, 40002400036]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39996000100, 39996800064, 39997600036, 39998400016, 39999200004, 40000000000, 40000800004, 40001600016, 40002400036, 40003200064]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39996800064, 39997600036, 39998400016, 39999200004, 40000000000, 40000800004, 40001600016, 40002400036, 40003200064, 40004000100]
 [4, 16, 36, 64, 100, 144, 196, 256, 324, 400  …  39997600036, 

In [30]:
@btime 1:100 .|> x->do_stuff2(1:(10_000+x))

  2.263 ms (201 allocations: 7.68 MiB)


100-element Array{Array{Int64,1},1}:
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29976, 29979, 29982, 29985, 29988, 29991, 29994, 29997, 30000, 30003]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29979, 29982, 29985, 29988, 29991, 29994, 29997, 30000, 30003, 30006]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29982, 29985, 29988, 29991, 29994, 29997, 30000, 30003, 30006, 30009]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29985, 29988, 29991, 29994, 29997, 30000, 30003, 30006, 30009, 30012]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29988, 29991, 29994, 29997, 30000, 30003, 30006, 30009, 30012, 30015]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29991, 29994, 29997, 30000, 30003, 30006, 30009, 30012, 30015, 30018]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29994, 29997, 30000, 30003, 30006, 30009, 30012, 30015, 30018, 30021]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  29997, 30000, 30003, 30006, 30009, 30012, 30015, 30018, 30021, 30024]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  30000, 30003, 30006, 30009, 30012,

In [31]:
@btime 1:100 .|> x->do_stuff2(1:(100_000+x))

  16.231 ms (201 allocations: 76.34 MiB)


100-element Array{Array{Int64,1},1}:
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299976, 299979, 299982, 299985, 299988, 299991, 299994, 299997, 300000, 300003]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299979, 299982, 299985, 299988, 299991, 299994, 299997, 300000, 300003, 300006]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299982, 299985, 299988, 299991, 299994, 299997, 300000, 300003, 300006, 300009]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299985, 299988, 299991, 299994, 299997, 300000, 300003, 300006, 300009, 300012]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299988, 299991, 299994, 299997, 300000, 300003, 300006, 300009, 300012, 300015]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299991, 299994, 299997, 300000, 300003, 300006, 300009, 300012, 300015, 300018]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299994, 299997, 300000, 300003, 300006, 300009, 300012, 300015, 300018, 300021]
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20  …  299997, 300000, 300003, 300006, 300009, 300012, 300015, 300018, 300021, 300

In [34]:
@code_native do_stuff1(1:100_000)

	.text
; ┌ @ In[25]:1 within `do_stuff1'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	subq	$192, %rsp
	vmovdqa	%xmm9, -64(%rbp)
	vmovdqa	%xmm8, -80(%rbp)
	vmovdqa	%xmm7, -96(%rbp)
	vmovdqa	%xmm6, -112(%rbp)
	movq	%rcx, %rsi
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%xmm0, -144(%rbp)
	movq	$0, -128(%rbp)
	movl	$jl_get_ptls_states, %eax
	callq	*%rax
	movq	%rax, %r14
; │┌ @ broadcast.jl:1238 within `broadcasted' @ broadcast.jl:1066
; ││┌ @ range.jl:565 within `first'
; │││┌ @ Base.jl:33 within `getproperty'
	movq	$4, -144(%rbp)
	movq	(%r14), %rax
	movq	%rax, -136(%rbp)
	leaq	-144(%rbp), %rax
	movq	%rax, (%r14)
; ││└└
; ││┌ @ int.jl:54 within `*'
	movq	(%rsi), %rdi
; ││└
; ││┌ @ range.jl:543 within `length'
; │││┌ @ checked.jl:222 within `checked_sub'
; ││││┌ @ checked.jl:194 within `sub_with_overflow'
	movq	8(%rsi), %rax
	movq	%rax, %rdx
	subq	%rdi, %rdx
; ││││└
; ││││ @ checked.jl:223 within `checked_sub'
	jo	L830
; │││└
; │││┌ @ checke